In [1]:
import pandas as pd


In [2]:
# 可转债交换套利
bond_diff_ids = ['127022', '127067']
# 可转债三低轮动
bond_trans_df = pd.read_csv('./strategy/bond_trans.csv')
bond_trans_ids = bond_trans_df['bond_id'].tolist()
for i in range(len(bond_trans_ids)):
    bond_trans_ids[i] = bond_trans_ids[i].split('.')[0]
# 绩优小市值轮动
small_stocks_df = pd.read_csv('./strategy/head_brief.csv')
small_stocks_ids = small_stocks_df['stock_id'].tolist()
for i in range(len(small_stocks_ids)):
    small_stocks_ids[i] = small_stocks_ids[i].split('.')[0]

In [10]:
from datetime import datetime

# 做账脚本
now = datetime.now()
curr_timestamp = f'{now.year}-{now.month}-{now.day}'
curr_path = f'./data/{curr_timestamp}-positions.xls'
curr_dataset = pd.read_excel(curr_path, dtype=str)
curr_dataset['市值'] = curr_dataset['市值'].astype('float')
curr_dataset['当日盈亏'] = curr_dataset['当日盈亏'].astype('float')
curr_dataset['日期'] = curr_dataset['证券代码'].astype('str')

def update_account(path, ids):
    # 可转债交换套利
    account_path = path
    account_dataset = pd.read_excel(account_path)
    curr_strategy_df = curr_dataset[curr_dataset['证券代码'].isin(ids)]
    # stat
    prev_total_mv = account_dataset.iloc[-1]['总市值']
    curr_total_mv = curr_strategy_df['市值'].sum()
    prev_ret = account_dataset.iloc[-1]['累计盈亏']
    today_ret = curr_strategy_df['当日盈亏'].sum()
    curr_ret = prev_ret + today_ret
    cash_change = curr_total_mv - (prev_total_mv + today_ret)
    today_ratio = today_ret / prev_total_mv
    prev_total_ratio = account_dataset.iloc[-1]['累计涨跌幅']
    curr_total_ratio = (1 + prev_total_ratio) * (1 + today_ratio) - 1
    net_value = 1 + curr_total_ratio
    # new data
    new_row = {
        '日期': curr_timestamp,
        '净值': net_value,
        '总市值': curr_total_mv,
        '当日涨跌幅': today_ratio,
        '累计涨跌幅': curr_total_ratio,
        '当日盈亏': today_ret,
        '累计盈亏': curr_ret,
        '出入金': cash_change
    }
    print(new_row)
    # concat
    account_dataset = account_dataset._append(new_row, ignore_index=True)
    account_dataset.to_excel(path, index=False)

update_account('./positions/可转债交换套利.xlsx', bond_diff_ids)
update_account('./positions/可转债三低轮动.xlsx', bond_trans_ids)
update_account('./positions/绩优小市值轮动.xlsx', small_stocks_ids)

{'日期': '2023-12-21', '净值': 0.9886037458057635, '总市值': 385478.32, '当日涨跌幅': 1.0376728960851274e-06, '累计涨跌幅': -0.01139625419423651, '当日盈亏': 0.4, '累计盈亏': -4443.650000000001, '出入金': -0.010000000009313226}
{'日期': '2023-12-21', '净值': 0.9996447938823202, '总市值': 1001895.0800000001, '当日涨跌幅': 0.010509098503854642, '累计涨跌幅': -0.00035520611767980714, '当日盈亏': 10423.76, '累计盈亏': -355.84000000000015, '出入金': -408.2399999999907}
{'日期': '2023-12-21', '净值': 0.9998824943880759, '总市值': 1500401.0, '当日涨跌幅': 0.010556722087605098, '累计涨跌幅': -0.00011750561192414022, '当日盈亏': 15671.0, '累计盈亏': -173.07999999999993, '出入金': 273.0}
